In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler  # Mudança para StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import traceback

# Configurar o TensorFlow para usar menos memória
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# --- Função para carregar dados ---
def carregar_dados_ticker(ticker, pasta_dados_tratados):
    """Carrega dados pré-processados, renomeia colunas e trata NaNs."""
    nome_arquivo = f"{ticker}_dados_macro_micro.csv"
    caminho_arquivo = os.path.join(pasta_dados_tratados, nome_arquivo)

    if not os.path.exists(caminho_arquivo):
        print(f"Erro: Arquivo não encontrado para {ticker} em {caminho_arquivo}")
        return None, None, None

    try:
        print(f"Carregando dados tratados de: {caminho_arquivo}")
        try:
            df = pd.read_csv(caminho_arquivo, index_col=0)
            df.index = pd.to_datetime(df.index)
        except Exception as e:
            print(f"Erro ao carregar arquivo: {e}")
            return None, None, None

        print("Colunas originais:", df.columns.tolist())

        # Identificar colunas de preço alvo
        target_col = None
        if 'Close_Target' in df.columns:
            target_col = 'Close_Target'
        elif 'Preço' in df.columns:
            target_col = 'Preço'
        else:
            print(f"Erro: Nenhuma coluna de preço alvo encontrada para {ticker}")
            return None, None, None

        # Identificar a coluna de feature de preço
        feature_cols = []
        price_feature_col = None
        
        if 'Close_Feature' in df.columns:
            price_feature_col = 'Close_Feature'
        elif 'Preço_anterior' in df.columns:
            price_feature_col = 'Preço_anterior'
        else:
            print(f"Aviso: Nenhuma coluna de preço anterior encontrada para {ticker}")
        
        if price_feature_col:
            feature_cols.append(price_feature_col)

        # Separar features macro e micro
        macro_cols = ['TaxaCambio', 'Selic', 'PIB', 'IPCA']
        micro_cols = ['ROA', 'ROE', 'Margem Líquida', 'P/L', 'VP']

        # Adicionar features disponíveis
        for col in macro_cols + micro_cols:
            if col in df.columns:
                feature_cols.append(col)

        # Verificar se temos features suficientes
        if len(feature_cols) < 2:  # Pelo menos uma feature além do preço
            print(f"Aviso: Poucas features encontradas para {ticker}: {feature_cols}")

        print(f"Features identificadas para {ticker}: {feature_cols}")
        print(f"Target identificado para {ticker}: {target_col}")

        # Tratar NaNs
        df = df[feature_cols + [target_col]].copy()
        df.dropna(subset=[target_col], inplace=True)  # Remove linhas onde o target é NaN
        
        # Garantir features numéricas
        for col in feature_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Preencher NaNs nas features
        df[feature_cols] = df[feature_cols].fillna(method='ffill')
        if df[feature_cols].isnull().any().any():
            df[feature_cols] = df[feature_cols].fillna(method='bfill')
        
        # Verificar se ainda há NaNs
        if df[feature_cols + [target_col]].isnull().any().any():
            print(f"Aviso: Ainda existem NaNs para {ticker}. Removendo linhas afetadas...")
            df.dropna(inplace=True)

        print(f"Dimensões finais do DataFrame para {ticker}: {df.shape}")
        if df.empty:
            print(f"Erro: DataFrame ficou vazio para {ticker} após limpeza.")
            return None, None, None

        return df, feature_cols, target_col

    except Exception as e:
        print(f"Erro CRÍTICO ao carregar/processar {caminho_arquivo}: {e}")
        traceback.print_exc()
        return None, None, None

# --- Função para criar janelas multivariadas com diferenciação ---
def criar_janelas_multivariadas_diff(features_array, target_array, janela, add_diff=True):
    """Cria janelas multivariadas com diferenciação para melhorar previsões"""
    X, y = [], []
    
    if add_diff and features_array.shape[1] > 0:
        # Adicionar colunas de diferenciação para cada feature
        diffs = np.diff(features_array, axis=0)
        # Concatenar com zeros no início para manter dimensões
        zeros_row = np.zeros((1, diffs.shape[1]))
        diffs_padded = np.vstack([zeros_row, diffs])
        
        # Combinar features originais com suas diferenciações
        features_array = np.concatenate([features_array, diffs_padded], axis=1)
    
    if len(features_array) <= janela:
        print(f"Aviso: dados insuficientes ({len(features_array)}) para janela ({janela}).")
        return np.array(X), np.array(y)
    
    for i in range(len(features_array) - janela):
        X.append(features_array[i:(i + janela), :])
        y.append(target_array[i + janela])  # Target já é T+1
    
    return np.array(X), np.array(y)

# --- Modelo LSTM melhorado com Residual Connections ---
def build_model_improved(input_shape, dropout_rate=0.2):
    """Constrói modelo LSTM com conexões residuais para evitar previsões travadas"""
    inputs = layers.Input(shape=input_shape)
    
    # Camada LSTM 1
    lstm1 = layers.LSTM(100, return_sequences=True)(inputs)
    lstm1 = layers.Dropout(dropout_rate)(lstm1)
    
    # Camada LSTM 2 com conexão residual
    lstm2 = layers.LSTM(100, return_sequences=False)(lstm1)
    lstm2 = layers.Dropout(dropout_rate)(lstm2)
    
    # Atenção à última dimensão da entrada para criar conexão residual
    input_flattened = layers.Flatten()(inputs)
    input_dense = layers.Dense(100)(input_flattened)
    
    # Combinar saída do LSTM e entrada (conexão residual)
    combined = layers.Add()([lstm2, input_dense])
    combined = layers.Activation('relu')(combined)
    
    # Camadas densas finais
    dense1 = layers.Dense(64, activation='relu')(combined)
    dense1 = layers.Dropout(dropout_rate/2)(dense1)
    
    # Saída
    outputs = layers.Dense(1)(dense1)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_squared_error'
    )
    return model

# --- Função principal para treinar e avaliar o modelo ---
def treinar_avaliar_modelo(ticker, janela, df_full, feature_cols, target_col, 
                           resultados_dir="resultados_lstm_macromicro_melhorado"):
    """Treina e avalia o modelo LSTM melhorado para um ticker e janela"""
    print(f"\n--- Iniciando Experimento: {ticker}, Janela {janela} ---")
    
    os.makedirs(resultados_dir, exist_ok=True)
    base_filename = f"{resultados_dir}/{ticker}_Janela_{janela}"
    metrics_path = f"{base_filename}_metrics.csv"
    grafico_path = f"{base_filename}_grafico_teste_final.png"
    previsoes_path = f"{base_filename}_previsoes_teste_final.csv"
    
    if os.path.exists(metrics_path):
        print(f"Resultados já existem para {ticker}, Janela {janela}. Pulando...")
        return
    
    # 1. Separar Dados Treino/Validação (2020-2022) e Teste (2023)
    start_date_val = "2020-01-01"
    end_date_val = "2022-12-31"
    start_date_test = "2023-01-01"
    end_date_test = "2023-12-31"
    
    try:
        df_val_train = df_full.loc[start_date_val:end_date_val].copy()
        df_test_final = df_full.loc[start_date_test:end_date_test].copy()
    except KeyError as e:
        print(f"Erro ao dividir dados por data para {ticker}: {e}.")
        if not df_full.empty: 
            print(f"Datas disponíveis: {df_full.index.min()} a {df_full.index.max()}")
        return
    
    if df_val_train.empty or df_test_final.empty:
        print(f"Erro: Período treino/val ou teste vazio para {ticker}.")
        return
    
    # 2. Preparar dados
    features_val_train = df_val_train[feature_cols].values
    target_val_train = df_val_train[target_col].values
    
    features_test = df_test_final[feature_cols].values
    target_test = df_test_final[target_col].values
    
    # 3. Escalonamento com StandardScaler (em vez de MinMaxScaler)
    scaler_features = StandardScaler()
    scaler_target = StandardScaler()
    
    # Ajustar escaladores apenas nos dados de treino
    scaled_features_val_train = scaler_features.fit_transform(features_val_train)
    target_val_train_reshaped = target_val_train.reshape(-1, 1)
    scaled_target_val_train = scaler_target.fit_transform(target_val_train_reshaped).flatten()
    
    # Transformar dados de teste
    scaled_features_test = scaler_features.transform(features_test)
    target_test_reshaped = target_test.reshape(-1, 1)
    scaled_target_test = scaler_target.transform(target_test_reshaped).flatten()
    
    # 4. Criar janelas com diferenciação
    X_train, y_train = criar_janelas_multivariadas_diff(
        scaled_features_val_train, 
        scaled_target_val_train, 
        janela,
        add_diff=True  # Adicionar colunas de diferenciação
    )
    
    X_test, y_test = criar_janelas_multivariadas_diff(
        scaled_features_test, 
        scaled_target_test, 
        janela,
        add_diff=True  # Adicionar colunas de diferenciação
    )
    
    if len(X_train) == 0 or len(X_test) == 0:
        print(f"Erro: Não foi possível criar janelas para {ticker}.")
        return
    
    # 5. Configurar validação
    val_split_idx = int(0.8 * len(X_train))
    X_train_final, y_train_final = X_train[:val_split_idx], y_train[:val_split_idx]
    X_val, y_val = X_train[val_split_idx:], y_train[val_split_idx:]
    
    # 6. Construir e treinar modelo
    input_shape = (janela, X_train.shape[2])
    model = build_model_improved(input_shape)
    
    # Callbacks para melhorar treinamento
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=20, 
        restore_best_weights=True, 
        verbose=1
    )
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-5,
        verbose=1
    )
    
    # Treinar modelo
    history = model.fit(
        X_train_final, y_train_final,
        validation_data=(X_val, y_val),
        epochs=200,
        batch_size=32,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
    # 7. Avaliar no conjunto de teste
    predictions_scaled = model.predict(X_test)
    predictions = scaler_target.inverse_transform(predictions_scaled).flatten()
    actual = scaler_target.inverse_transform(y_test.reshape(-1, 1)).flatten()
    
    # 8. Calcular métricas
    mae = mean_absolute_error(actual, predictions)
    mse = mean_squared_error(actual, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(actual, predictions)
    
    # 9. Verificar variação das previsões (para detectar previsões travadas)
    var_real = np.std(actual) / np.mean(actual) if np.mean(actual) != 0 else 0
    var_pred = np.std(predictions) / np.mean(predictions) if np.mean(predictions) != 0 else 0
    prop_var = var_pred / var_real if var_real != 0 else 0
    travado = "Sim" if prop_var < 0.3 else "Não"
    
    print("\nResultados da Avaliação:")
    print(f"MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")
    print(f"Variação Real: {var_real:.4f}, Variação Prevista: {var_pred:.4f}")
    print(f"Previsões travadas? {travado}")
    
    # 10. Salvar resultados
    # Métricas
    metrics_df = pd.DataFrame([{
        'Ticker': ticker,
        'Janela': janela,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2,
        'Var_Real': var_real,
        'Var_Pred': var_pred,
        'Prop_Var': prop_var,
        'Travado': travado
    }])
    metrics_df.to_csv(metrics_path, index=False)
    
    # Datas para as previsões
    datas_teste = df_test_final.index[janela:janela+len(actual)]
    
    if len(datas_teste) == len(actual):
        # Salvar previsões
        previsoes_df = pd.DataFrame({
            'Data': datas_teste,
            'Preço Real': actual,
            'Preço Previsto': predictions
        })
        previsoes_df.to_csv(previsoes_path, index=False)
        
        # Gerar gráfico
        plt.figure(figsize=(14, 7))
        plt.plot(previsoes_df['Data'], previsoes_df['Preço Real'], label='Real', color='blue', linewidth=1.5)
        plt.plot(previsoes_df['Data'], previsoes_df['Preço Previsto'], label='Previsto', color='orange', linestyle='--', linewidth=1.5)
        plt.title(f'Preços Reais vs Previstos - {ticker} (Janela {janela})')
        plt.xlabel('Data')
        plt.ylabel('Preço')
        plt.legend()
        plt.grid(alpha=0.4)
        plt.tight_layout()
        plt.savefig(grafico_path)
        plt.close()
    
    print(f"--- Experimento concluído: {ticker}, Janela {janela} ---")
    return metrics_df

# --- Função para rodar experimentos em todos os tickers ---
def rodar_experimentos_macromicro_melhorados():
    pasta_dados = r"C:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\dados_unificados"
    resultados_dir = r"C:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\resultados_lstm_macromicro_melhorado"
    os.makedirs(resultados_dir, exist_ok=True)
    
    # Lista de tickers
    tickers = ["GGBR3.SA"]
    
    # Janelas a testar
    janelas = [1, 2, 3, 4, 5]
    
    resultados_consolidados = []
    
    for ticker in tickers:
        df_ticker_full, feature_cols, target_col = carregar_dados_ticker(ticker, pasta_dados)
        
        if df_ticker_full is None:
            print(f"Pulando ticker {ticker} devido a erro no carregamento.")
            continue
        
        for janela in janelas:
            try:
                metrics = treinar_avaliar_modelo(ticker, janela, df_ticker_full, feature_cols, target_col, resultados_dir)
                if metrics is not None:
                    resultados_consolidados.append(metrics)
            except Exception as e:
                print(f"Erro ao processar {ticker}, Janela {janela}: {e}")
                traceback.print_exc()
    
    # Consolidar resultados
    if resultados_consolidados:
        df_resultados = pd.concat(resultados_consolidados)
        df_resultados.to_csv(f"{resultados_dir}/resultados_consolidados.csv", index=False)
        print(f"\nResultados consolidados salvos em {resultados_dir}/resultados_consolidados.csv")

if __name__ == "__main__":
    # Desativar mensagens de aviso do TensorFlow
    tf.get_logger().setLevel('ERROR')
    
    print("Iniciando experimentos com modelos melhorados para features macro/micro...")
    rodar_experimentos_macromicro_melhorados()
    print("\nTodos experimentos concluídos!")


Iniciando experimentos com modelos melhorados para features macro/micro...
Carregando dados tratados de: C:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\dados_unificados\GGBR3.SA_dados_macro_micro.csv
Colunas originais: ['Preço', 'Close_Feature', 'Close_Target', 'ROA', 'ROE', 'Margem Líquida', 'P/L', 'VP', 'Preço_anterior', 'TaxaCambio', 'Selic', 'PIB', 'IPCA', "('GGBR3.SA_Close', 'GGBR3.SA')"]
Features identificadas para GGBR3.SA: ['Close_Feature', 'TaxaCambio', 'Selic', 'PIB', 'IPCA', 'ROA', 'ROE', 'Margem Líquida', 'P/L', 'VP']
Target identificado para GGBR3.SA: Close_Target
Dimensões finais do DataFrame para GGBR3.SA: (1460, 11)

--- Iniciando Experimento: GGBR3.SA, Janela 1 ---
Epoch 1/200


C:\Users\leona\AppData\Local\Temp\ipykernel_14240\1233118696.py:93: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[feature_cols] = df[feature_cols].fillna(method='ffill')
C:\Users\leona\AppData\Local\Temp\ipykernel_14240\1233118696.py:95: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[feature_cols] = df[feature_cols].fillna(method='bfill')


28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 0.6097 - val_loss: 0.0461 - learning_rate: 0.0010
Epoch 2/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0708 - val_loss: 0.0432 - learning_rate: 0.0010
Epoch 3/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0392 - val_loss: 0.0538 - learning_rate: 0.0010
Epoch 4/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0427 - val_loss: 0.0716 - learning_rate: 0.0010
Epoch 5/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0339 - val_loss: 0.0601 - learning_rate: 0.0010
Epoch 6/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0289 - val_loss: 0.0592 - learning_rate: 0.0010
Epoch 7/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0258 - val_loss: 0.0589 - learning_rate: 0.0010
Epoch 8/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0222 - val_loss: 0.0455 - learning_rate: 0.0010
Epoch 9/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0224 - val_loss: 0.0572 - learning_rate: 0.0010
Epoch 10/200
28/28 ━━━━━━━━━━━